In [1]:
import requests
import pandas as pd
import csv
import datetime
import dateutil
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta, FR
import json
from pathlib import Path
from difflib import SequenceMatcher
import operator
from langdetect import detect
import re
import plotly.plotly as py
import plotly.graph_objs as go
# Scientific libraries
from numpy import arange,array,ones
from scipy import stats

key = '375a4858ec65578a421621d1f5a0cc53'

In [2]:
#save songs
def save_songs(songs):
    if songs != {}:
        with open('songs.json', 'w') as fp:
            json.dump(songs, fp)
        
def load_songs():
    #load songs
    with open('songs.json', 'r') as fp:
        songs = json.load(fp)
        
    return songs
       
#compare similarity of two strings
def similar(a, b):
    sep = "feat"
    arest = a.split(sep, 1)[0]
    brest = b.split(sep, 1)[0]
    return SequenceMatcher(None, arest, brest).ratio()

#not currently using this
def get_lang(str1, str2):
    if (type(str1) == str):     
        if (re.search('[a-zA-Z]', str1)):
            return detect(str1)
  
    return 'X'

#get language/lyric information for a single track
def get_track(song, artist):
    songs = load_songs()
    search_url = 'http://api.musixmatch.com/ws/1.1/track.search'
    params = {'q_track' : song, 'page_size' : '30', 'page' : '1', 's_track_rating' : 'desc', 'apikey': key}
    resp = requests.get(search_url, params=params)
    tracks = json.loads(resp.text)['message']['body']['track_list']

    artist_results = {}

    index = 0
    for track in tracks:
        track_id = track['track']['track_id']
        artist_results[index] = similar(track['track']['artist_name'], artist)
        index = index + 1

    sorted_index = sorted(artist_results.items(), key=operator.itemgetter(1), reverse=True)
    
    if ((len(sorted_index) > 0) and (sorted_index[0][1] > .6)):
        winningIndex = sorted_index[0][0]
        track_id = tracks[winningIndex]['track']['track_id']
        print(track_id)

        search_url = 'http://api.musixmatch.com/ws/1.1/track.lyrics.get'
        params = {'track_id' : track_id, 'apikey': key}
        resp = requests.get(search_url, params=params)

        body = json.loads(resp.text)['message']['body']
        
        if ('lyrics' in body):
            lyrics = body['lyrics']
            lyrics_body  = lyrics['lyrics_body']
            lyrics_language = lyrics['lyrics_language']

            id = (song + " by " + artist)
            song_info = {}
            song_info['song'] = song
            song_info['artist'] = artist
            song_info['lyrics_body'] = lyrics['lyrics_body'].replace('\n', ' ')[:-59]
            song_info['lyrics_language'] = lyrics['lyrics_language']
            songs[id] = song_info
        else:
            print("empty resp")
            id = (song + " by " + artist)
            song_info = {}
            song_info['song'] = song
            song_info['artist'] = artist
            song_info['lyrics_body'] = "no lyrics"
            lg = "X"
            song_info['lyrics_language'] = lg
            songs[id] = song_info
    else:
        id = (song + " by " + artist)
        song_info = {}
        song_info['song'] = song
        song_info['artist'] = artist
        song_info['lyrics_body'] = "no lyrics"
        lg = "X"
        song_info['lyrics_language'] = lg
        songs[id] = song_info
        
    save_songs(songs)
       
#go through every song in a file
def get_songs(df, fileName):
    songs = load_songs()
    languages = []
    lyrics = []
    for songTitle, artist in zip(df['Track Name'], df['Artist']):
        songTitle = str(songTitle)
        id = (songTitle + ' by ' + artist) 
        print(id)
        #if we do not alreay have the songs language and info archived, then query for it
        if id not in songs:
            get_track(songTitle, artist)
        else:
            print("already have")
        songs = load_songs()
        songInfo = songs[id]
        lyrics.append(songInfo['lyrics_body'])
        languages.append(songInfo['lyrics_language'])
        
    #add new info to dataframe
    df['language'] = languages
    df['lyrics'] = lyrics
    
    df.to_csv(fileName)
    
#create urls and download files for a country and the last x weeks
def download_charts(country, weeks):
    
    #find last fridays date
    last_friday = datetime.now() + relativedelta(weekday=FR(-1))

    if (datetime.now().strftime('%Y-%m-%d') == last_friday.strftime('%Y-%m-%d')):
        last_friday = datetime.now() + relativedelta(weekday=FR(-1))

    files = []
    urls = []
    newer_date = last_friday - timedelta(days=7)
    
    #create uls by counting back a friday at a time
    for i in range(weeks):
        older_date = newer_date - timedelta(days=7)
        newer_str = newer_date.strftime('%Y-%m-%d')
        older_str = older_date.strftime('%Y-%m-%d')
        newer_date = older_date
        url = 'https://spotifycharts.com/regional/' + country + '/weekly/' + older_str + '--' + newer_str + '/download'
        file_name = country + "_" + older_str + '--' + newer_str + '.csv'
#         print(url)
#         print(file_name)
        urls.append(url)
        files.append(file_name)
        
    #if we do not already have the file download it
    for file, url in zip(files,urls):
        my_file = Path(file)
        if not my_file.is_file():
            resp = requests.get(url)
            with open(file, 'w') as f:
                writer = csv.writer(f)
                reader = csv.reader(resp.text.splitlines())

                for row in reader:
                    writer.writerow(row)
                
    return files

#take file and add language/lyric info if needed
def add_language(files):

    for file in files:
        
        #load and reformat this file
        df = pd.read_csv(file,skiprows=[0])
        if (list(df) != ['Position', 'Track Name', 'Artist', 'Streams', 'URL']):
            df = pd.read_csv(file,skiprows=[])
            if 'Unnamed: 0' in list(df):
                df = df.drop('Unnamed: 0', axis=1)


        df.Artist = df.Artist.astype(str)
        
        #use api and add language/lyrics only if you do not already have them
        if ('language' not in list(df)):
            get_songs(df, file) 
            songs = load_songs()
            print(len(songs))
        
def get_data(country, weeks):
       
    #download chart files
    files = download_charts(country, weeks)   
    
    #add language information to files
    add_language(files)

    #analyze language for every week
    country = []
    week = []
    english_percent = []
    spanish_percent = []

    for file in files:
        df = pd.read_csv(file,skiprows=[0])
        if (list(df) != ['Position', 'Track Name', 'Artist', 'Streams', 'URL']):
            df = pd.read_csv(file,skiprows=[])
            if 'Unnamed: 0' in list(df):
                df = df.drop('Unnamed: 0', axis=1)


        df.Artist = df.Artist.astype(str)
        lang = {}
        for l in df['language']:
            if l in lang:
                lang[l] = lang[l] + 1
            else:
                lang[l] = 1

        eng = 0
        if "en" in lang:
            eng = lang['en']

        esp = 0
        if "es" in lang:
            esp = lang['es']

        total = eng + esp
        english_percent.append(round((eng/total),2))
        spanish_percent.append(round((esp/total),2))
        country.append(file[0:2])
        week.append(file[3:25])

    data = pd.DataFrame(
            {'country': country,
             'week': week,
             "english_percent": english_percent,
             "spanish_percent": spanish_percent
            })
    
    return data

In [3]:
us = get_data('us', 52)
us.sort_values(by=['week'], ascending=False)
us.head(52)

,country,english_percent,spanish_percent,week
0,us,0.97,0.03,2018-07-20--2018-07-27
1,us,0.98,0.02,2018-07-13--2018-07-20
2,us,0.97,0.03,2018-07-06--2018-07-13
3,us,0.98,0.02,2018-06-29--2018-07-06
4,us,0.99,0.01,2018-06-22--2018-06-29
5,us,0.99,0.01,2018-06-15--2018-06-22
6,us,0.99,0.01,2018-06-08--2018-06-15
7,us,0.99,0.01,2018-06-01--2018-06-08
8,us,0.99,0.01,2018-05-25--2018-06-01
9,us,0.99,0.01,2018-05-18--2018-05-25


In [4]:
do = get_data('do', 52)
do.sort_values(by=['week'], ascending=False)
do.head(52)

,country,english_percent,spanish_percent,week
0,do,0.22,0.78,2018-07-20--2018-07-27
1,do,0.21,0.79,2018-07-13--2018-07-20
2,do,0.23,0.77,2018-07-06--2018-07-13
3,do,0.31,0.69,2018-06-29--2018-07-06
4,do,0.29,0.71,2018-06-22--2018-06-29
5,do,0.28,0.72,2018-06-15--2018-06-22
6,do,0.24,0.76,2018-06-08--2018-06-15
7,do,0.28,0.72,2018-06-01--2018-06-08
8,do,0.22,0.78,2018-05-25--2018-06-01
9,do,0.22,0.78,2018-05-18--2018-05-25


In [5]:
mx = get_data('mx', 52)
mx.sort_values(by=['week'], ascending=False)
mx.head(52)

,country,english_percent,spanish_percent,week
0,mx,0.25,0.75,2018-07-20--2018-07-27
1,mx,0.22,0.78,2018-07-13--2018-07-20
2,mx,0.21,0.79,2018-07-06--2018-07-13
3,mx,0.27,0.73,2018-06-29--2018-07-06
4,mx,0.26,0.74,2018-06-22--2018-06-29
5,mx,0.26,0.74,2018-06-15--2018-06-22
6,mx,0.24,0.76,2018-06-08--2018-06-15
7,mx,0.26,0.74,2018-06-01--2018-06-08
8,mx,0.27,0.73,2018-05-25--2018-06-01
9,mx,0.26,0.74,2018-05-18--2018-05-25


In [6]:
mx = get_data('cr', 52)
mx.sort_values(by=['week'], ascending=False)
mx.head(52)

,country,english_percent,spanish_percent,week
0,cr,0.38,0.62,2018-07-20--2018-07-27
1,cr,0.41,0.59,2018-07-13--2018-07-20
2,cr,0.38,0.62,2018-07-06--2018-07-13
3,cr,0.37,0.63,2018-06-29--2018-07-06
4,cr,0.40,0.60,2018-06-22--2018-06-29
5,cr,0.42,0.58,2018-06-15--2018-06-22
6,cr,0.40,0.60,2018-06-08--2018-06-15
7,cr,0.42,0.58,2018-06-01--2018-06-08
8,cr,0.40,0.60,2018-05-25--2018-06-01
9,cr,0.39,0.61,2018-05-18--2018-05-25


In [7]:
ar = get_data('ar', 52)
ar.sort_values(by=['week'], ascending=False)
ar.head(52)

,country,english_percent,spanish_percent,week
0,ar,0.18,0.82,2018-07-20--2018-07-27
1,ar,0.20,0.80,2018-07-13--2018-07-20
2,ar,0.17,0.83,2018-07-06--2018-07-13
3,ar,0.18,0.82,2018-06-29--2018-07-06
4,ar,0.20,0.80,2018-06-22--2018-06-29
5,ar,0.18,0.82,2018-06-15--2018-06-22
6,ar,0.13,0.87,2018-06-08--2018-06-15
7,ar,0.15,0.85,2018-06-01--2018-06-08
8,ar,0.15,0.85,2018-05-25--2018-06-01
9,ar,0.14,0.86,2018-05-18--2018-05-25


In [8]:
gt = get_data('gt', 52)
gt.sort_values(by=['week'], ascending=False)
gt.head(52)

,country,english_percent,spanish_percent,week
0,gt,0.26,0.74,2018-07-20--2018-07-27
1,gt,0.29,0.71,2018-07-13--2018-07-20
2,gt,0.31,0.69,2018-07-06--2018-07-13
3,gt,0.29,0.71,2018-06-29--2018-07-06
4,gt,0.28,0.72,2018-06-22--2018-06-29
5,gt,0.30,0.70,2018-06-15--2018-06-22
6,gt,0.28,0.72,2018-06-08--2018-06-15
7,gt,0.29,0.71,2018-06-01--2018-06-08
8,gt,0.27,0.73,2018-05-25--2018-06-01
9,gt,0.26,0.74,2018-05-18--2018-05-25


In [9]:
co = get_data('co', 52)
co.sort_values(by=['week'], ascending=False)
co.head(52)

,country,english_percent,spanish_percent,week
0,co,0.25,0.75,2018-07-20--2018-07-27
1,co,0.26,0.74,2018-07-13--2018-07-20
2,co,0.24,0.76,2018-07-06--2018-07-13
3,co,0.23,0.77,2018-06-29--2018-07-06
4,co,0.23,0.77,2018-06-22--2018-06-29
5,co,0.24,0.76,2018-06-15--2018-06-22
6,co,0.20,0.80,2018-06-08--2018-06-15
7,co,0.20,0.80,2018-06-01--2018-06-08
8,co,0.23,0.77,2018-05-25--2018-06-01
9,co,0.25,0.75,2018-05-18--2018-05-25


In [10]:
glbl = get_data('global', 52)
glbl.sort_values(by=['week'], ascending=False)
glbl.head(52)

,country,english_percent,spanish_percent,week
0,gl,0.81,0.19,bal_2018-07-20--2018-0
1,gl,0.83,0.17,bal_2018-07-13--2018-0
2,gl,0.83,0.17,bal_2018-07-06--2018-0
3,gl,0.83,0.17,bal_2018-06-29--2018-0
4,gl,0.85,0.15,bal_2018-06-22--2018-0
5,gl,0.85,0.15,bal_2018-06-15--2018-0
6,gl,0.84,0.16,bal_2018-06-08--2018-0
7,gl,0.83,0.17,bal_2018-06-01--2018-0
8,gl,0.85,0.15,bal_2018-05-25--2018-0
9,gl,0.84,0.16,bal_2018-05-18--2018-0


In [11]:
import pandas as pd

def get_all_data(countries):
    #find last fridays date
    last_friday = datetime.now() + relativedelta(weekday=FR(-1))

    if (datetime.now().strftime('%Y-%m-%d') == last_friday.strftime('%Y-%m-%d')):
        last_friday = datetime.now() + relativedelta(weekday=FR(-1))

    all_pds = []
    files = []
    urls = []
    country = []
    week = []
    newer_date = last_friday - timedelta(days=7)

    #create uls by counting back a friday at a time
    for i in range(52):
        dfs = []
        first = True
        older_date = newer_date - timedelta(days=7)
        newer_str = newer_date.strftime('%Y-%m-%d')
        older_str = older_date.strftime('%Y-%m-%d')
        newer_date = older_date
        for c in countries:
            file_name = c + "_" + older_str + '--' + newer_str + '.csv'
            df = pd.read_csv(file_name,skiprows=[0])
            if (list(df) != ['Position', 'Track Name', 'Artist', 'Streams', 'URL']):
                df = pd.read_csv(file_name,skiprows=[])
                if 'Unnamed: 0' in list(df):
                    df = df.drop('Unnamed: 0', axis=1)


            df.Artist = df.Artist.astype(str)
            dfs.append(df)
            if first:
                first = False
                country.append("all")
                week.append(file_name[3:25])
        whole=pd.concat(dfs)   
        all_pds.append(whole)  

    #analyze language for every week
    english_percent = []
    spanish_percent = []

    for df in all_pds:
        if (list(df) != ['Position', 'Track Name', 'Artist', 'Streams', 'URL']):
            if 'Unnamed: 0' in list(df):
                df = df.drop('Unnamed: 0', axis=1)

        lang = {}
        for l in df['language']:
            if l in lang:
                lang[l] = lang[l] + 1
            else:
                lang[l] = 1

        eng = 0
        if "en" in lang:
            eng = lang['en']

        esp = 0
        if "es" in lang:
            esp = lang['es']

        total = eng + esp
        english_percent.append(round((eng/total),2))
        spanish_percent.append(round((esp/total),2))

    data = pd.DataFrame(
            {'country': country,
             'week': week,
             "english_percent": english_percent,
             "spanish_percent": spanish_percent
            })
    
    return data

In [12]:
all_countries = get_all_data(["mx","gt","ar","do", "co"])
all_countries.head(52)
all_countries.to_csv("all_countries.csv")

In [13]:
# Create a trace
trace = go.Scatter(
    x = us['week'],
    y = us['spanish_percent']
)

data = [trace]

layout = dict(title = 'Percent of Spanish songs in Spotifys US weekly top 200 Chart',
              xaxis = dict(title = 'Week', autorange='reversed'),
              yaxis = dict(title = 'Percent of Spanish songs', tickformat = '.0%'),
              )

fig = dict(data=data, layout=layout)
py.iplot(figure_or_data=fig, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jgluck321/0 or inside your plot.ly account where it is named 'basic-line'


In [14]:
# Create a trace
trace = go.Scatter(
    x = do['week'],
    y = do['spanish_percent']
)

data = [trace]

layout = dict(title = 'Percent of Spanish songs in Spotifys Dominican Republic weekly top 200 Chart',
              xaxis = dict(title = 'Week', autorange='reversed'),
              yaxis = dict(title = 'Percent of Spanish songs', tickformat = '.0%'),
              )

fig = dict(data=data, layout=layout)
py.iplot(figure_or_data=fig, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jgluck321/0 or inside your plot.ly account where it is named 'basic-line'


In [15]:
# Create a trace
trace = go.Scatter(
    x = co['week'],
    y = co['spanish_percent']
)

data = [trace]

layout = dict(title = 'Percent of Spanish songs in Spotifys Colombia weekly top 200 Chart',
              xaxis = dict(title = 'Week', autorange='reversed'),
              yaxis = dict(title = 'Percent of Spanish songs', tickformat = '.0%'),
              )

fig = dict(data=data, layout=layout)
py.iplot(figure_or_data=fig, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jgluck321/0 or inside your plot.ly account where it is named 'basic-line'


In [16]:
# Create a trace
trace = go.Scatter(
    x = mx['week'],
    y = mx['spanish_percent']
)

data = [trace]

layout = dict(title = 'Percent of Spanish songs in Spotifys Mexico weekly top 200 Chart',
              xaxis = dict(title = 'Week', autorange='reversed'),
              yaxis = dict(title = 'Percent of Spanish songs', tickformat = '.0%'),
              )

fig = dict(data=data, layout=layout)
py.iplot(figure_or_data=fig, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jgluck321/0 or inside your plot.ly account where it is named 'basic-line'


In [17]:
# Create a trace
trace = go.Scatter(
    x = ar['week'],
    y = ar['spanish_percent']
)

data = [trace]

layout = dict(title = 'Percent of Spanish songs in Spotifys Argentina weekly top 200 Chart',
              xaxis = dict(title = 'Week', autorange='reversed'),
              yaxis = dict(title = 'Percent of Spanish songs', tickformat = '.0%'),
              )

fig = dict(data=data, layout=layout)
py.iplot(figure_or_data=fig, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jgluck321/0 or inside your plot.ly account where it is named 'basic-line'


In [18]:
# Create a trace
trace = go.Scatter(
    x = gt['week'],
    y = gt['spanish_percent']
)

data = [trace]

layout = dict(title = 'Percent of Spanish songs in Spotifys Guatemala weekly top 200 Chart',
              xaxis = dict(title = 'Week', autorange='reversed'),
              yaxis = dict(title = 'Percent of Spanish songs', tickformat = '.0%'),
              )

fig = dict(data=data, layout=layout)
py.iplot(figure_or_data=fig, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jgluck321/0 or inside your plot.ly account where it is named 'basic-line'


In [19]:
# Create a trace
trace = go.Scatter(
    x = glbl['week'],
    y = glbl['spanish_percent']
)

data = [trace]

layout = dict(title = 'Percent of Spanish songs in Spotifys Global weekly top 200 Chart',
              xaxis = dict(title = 'Week', autorange='reversed'),
              yaxis = dict(title = 'Percent of Spanish songs', tickformat = '.0%'),
              )

fig = dict(data=data, layout=layout)
py.iplot(figure_or_data=fig, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jgluck321/0 or inside your plot.ly account where it is named 'basic-line'


In [20]:
# Create a trace
trace = go.Scatter(
    x = all_countries['week'],
    y = all_countries['spanish_percent'],
    name='Spanish Percent',
)

xi = arange(0,len(all_countries))

# Generated linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,all_countries['spanish_percent'])
line = slope*xi+intercept

trace2 = go.Scatter(
                  x=all_countries['week'],
                  y=line,
                  mode='lines',
                  marker=go.Marker(color='rgb(31, 119, 180)'),
                  name='Fit',
                  line = dict(
                    color = ('rgb(205, 12, 24)'),
                    width = 2)
                  )

data = [trace, trace2]

layout = dict(title = 'Percent of Spanish songs in Spotifys Combined Spanish Speaking Countries weekly top 200 Chart',
              xaxis = dict(title = '', autorange='reversed'),
              yaxis = dict(title = 'Percent of Spanish songs', tickformat = '.0%'),
              )

fig = dict(data=data, layout=layout)
py.iplot(figure_or_data=fig, filename='basic-line')

/Users/jgluck/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.




High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jgluck321/0 or inside your plot.ly account where it is named 'basic-line'
